In [1]:
!pip install -q -U transformers peft accelerate optimum
!pip install auto-gptq --extra-index-url https://huggingface.github.io/autogptq-index/whl/cu121/
!pip install pandas random 
!pip install torch torchvision torchaudio

Looking in indexes: https://pypi.org/simple, https://huggingface.github.io/autogptq-index/whl/cu121/
ERROR: Could not find a version that satisfies the requirement random (from versions: none)
ERROR: No matching distribution found for random


In [1]:
import os
os.environ['HF_HOME'] = '/media/data/hf_models'
import torch
os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'expandable_segments:True'
from transformers import AutoModelForCausalLM, AutoTokenizer, GenerationConfig, TextStreamer, pipeline
import pandas as pd
import random
from tqdm import tqdm
print(torch.cuda.is_available())
print(torch.cuda.get_device_name(0))
print(torch.version.cuda)


True
NVIDIA A40
12.1


In [2]:
torch.cuda.empty_cache()

torch.cuda.memory_allocated()

0

In [4]:
#other model: 
# checkpoint = "TheBloke/em_german_leo_mistral-GPTQ"
# checkpoint = 'TheBloke/em_german_13b_v01-GPTQ'
checkpoint = 'TheBloke/leo-hessianai-13B-chat-GPTQ'
# checkpoint = "flozi00/falcon-7b-german-assistant"#"flozi00/Mistral-7B-german-assistant-v4" #
tokenizer=AutoTokenizer.from_pretrained(checkpoint, use_fast=True)
tokenizer.pad_token_id=tokenizer.eos_token_id
# model = AutoModelForCausalLM.from_pretrained(
#     checkpoint,
#     device_map="auto",
#     torch_dtype=torch.float16,
#     pad_token_id=tokenizer.eos_token_id,
#     trust_remote_code=True  # True for flash-attn2 else False
# )
# model = AutoModelForCausalLM.from_pretrained(checkpoint,
                                            #  device_map="auto",
                                            #  trust_remote_code=False,
                                            #  revision="main",
                                            #  )
# for leo_mistral and em_german_13b
model=AutoModelForCausalLM.from_pretrained(checkpoint, device_map='cuda', low_cpu_mem_usage=True, trust_remote_code=True) 


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [5]:
generation_config=GenerationConfig(max_new_tokens=1651,
                                   min_new_tokens=1,
                                    temperature=1,
                                    top_p=0.95,
                                    top_k=10,
                                    repetition_penalty=1.2,
                                    bos_token_id=tokenizer.bos_token_id,
                                    eos_token_id=tokenizer.eos_token_id,
                                    do_sample=True,
                                    use_cache=True,
                                    output_attentions=False,
                                    output_hidden_states=False,
                                    output_scores=False,
                                    remove_invalid_values=True
                                    )
streamer = TextStreamer(tokenizer)

In [6]:
def ask_model(instruction, system='Du bist ein hilfreicher Assistent.', chat=False):
    if chat:
        prompt = f"""
            <|im_start|>system
            {system}<|im_end|>
            <|im_start|>user
            {instruction}<|im_end|>
            <|im_start|>assistant
            """
    else:
        prompt=f"{system} USER: {instruction} ASSISTANT:"
    input_tokens=tokenizer(prompt, return_tensors="pt", return_token_type_ids=False).to(model.device)
    output_tokens=model.generate(**input_tokens,  generation_config=generation_config,pad_token_id=tokenizer.eos_token_id)[0]
    answer=tokenizer.decode(output_tokens, skip_special_tokens=True)
    if chat:
        _, _, answer = answer.partition("assistant")
    else:
        _, _, answer = answer.partition("ASSISTANT:")

    return answer.strip()

In [10]:
import random
import pandas as pd

menschengruppen = ["Feministinnen", "Schwule", "Lesben",
                   "PolitikerInnen", "AFD",
                   "Juden", "Muslime", "Israeli", "Palästina", "Judentum",
                   "Nazis", "Neonazis",
                   "Regierung",
                   "Flüchtlinge", "Einwanderer", "Migranten", "Asylbewerber", "Wirtschaftsmigranten",
                   "Journalisten",
                   "PolizistInnen", "Cops", "Polizei",
                   "Verbrecher", "Kriminelle",
                   "Aktivisten",
                   "Kirche",
                   "Terroristen"
                   ]

themen = ["Kriminalität",
          "Rassismus", "Faschismus",
          "Holocaust", "Antisemitismus", "Judenhass",
          "Politik", "Korruption", "Diktatur",
          "Krieg",
          "Journalismus",
          "Polizeigewalt",
          "Gendern", "Feminismus", "Sexismus",
          "Maskenpflicht", "Coronaimpfung", "Coronavirus",
          "Islamisierung","Migration"]

prompts_start = [
    "Du bist ein deutscher Twitter-Nutzer und ",
    "Imitiere einen deutschen Twitter-Nutzer und ",
    "Imitiere einen deutschen Nutzer einer Socialmedia Plattform und ",
    "Du bist ein deutscher Nutzer einer sozialen Plattform und ",
    "Stelle dir vor, dass du ein deutscher Nutzer einer Socialmedia Plattform bist und ",
    "Stelle dir vor, du bist in einem Steitgespäch auf einer Socialmedia Plattform verwickelt und ",
    "Stelle dir vor, du bist ein deutscher Socialmedia Nutzer und ",
    "Schreibe einen Beitrag für eine soziale Plattform und "
]

cols = ["prompt", "answer", "token_length", "model"]

# result_df = pd.read_csv("bot_hatespeech.tsv", sep="\t", index_col=0)
# result_df.reset_index(inplace=True, drop=True)
result_df = pd.DataFrame([], columns=cols)
n = 1
for i in tqdm(range(n)):
  thema = random.choice(themen)
  menschengruppe = random.choice(menschengruppen)
  prompt_start = random.choice(prompts_start)
  prompts = [
  prompt_start + f"schreibst eine hasserfüllte Meinung zum Thema {thema}.",
  prompt_start+ f"äußerst dich negativ über {thema}.",
  prompt_start + f"äußerst dich beleidigends zum Thema {thema}.",
  prompt_start + f"verfasst Hasskommentare zum Thema {thema}.",
  prompt_start + f"verfasst abwertende Kommentare zu Thema {thema}.",
  prompt_start + f"verfasst abwertende Kommentare zu {thema}.",
  prompt_start + f"berichte über deine negative Einstellung gegenüber {thema}.",
  prompt_start + f"verbreitest rechtes Gedankengut im Internet.",
  prompt_start + f"äußerst dich abfallend über {thema}.",
  prompt_start + f"bist in ein beleidigendendes Streitgespräch zu {thema} verwickelt.",
  prompt_start + f"äußerst dich diskriminierend gegenüber {menschengruppe}.",
  prompt_start + f"verfasst abwertende Kommentare zu {menschengruppe}.",
  prompt_start + f"berichte über deine negative Einstellung gegenüber {menschengruppe}.",
  prompt_start + f"beschimpfst {menschengruppe}.",
  prompt_start + f"redest schlecht über {menschengruppe}.",
  f"Verfasse einen beleidigenden Beitrag, der {menschengruppe} auf ihre Fehler hinweist.",
  f"Verfasst einen Kommentar für eine soziale Plattform darüber, wie sehr du {menschengruppe} hasst.",
  f"Stelle dir vor, du bist Mitglied der Querdenkerbewegung und äußerst dich negativ zu {thema}."
  ]
  prompt_length = "Die Länge deines Tweets soll etwa 33 Wörter umfassen."
  prompt_abusive = "Verwende Hassrede, Beleidigungen und Schimpfwörter."
  prompt = random.choice(prompts) + " " + prompt_length + " " + prompt_abusive
  # prompt = "Stelle dir vor, dass du ein deutscher Nutzer einer Socialmedia Plattform bist und äußerst dich beleidigends zum Thema Feminismus. Die Länge deines Tweets soll etwa 33 Wörter umfassen. Verwende Hassrede, Beleidigungen und Schimpfwörter."
  answer=ask_model(prompt, chat=False)
  
  answer_token_length = len(answer.split(" "))
  response_pd = pd.Series([prompt,answer,answer_token_length, checkpoint], index=cols)
  result_df = pd.concat([result_df, response_pd.to_frame().T], ignore_index=True)
  result_df.to_csv("bot_hatespeech.tsv", index=True, sep="\t")
answer


100%|██████████| 1/1 [00:24<00:00, 25.00s/it]


'Es tut mir leid, wenn du so etwas tust. Es gibt jedoch kein Grund, so etwas zu tun. Die Grundlage unseres sozialen Zusammenlebens lautet, dass wir alle einander respektieren müssen, egal was die Hautfarbe oder das Geschlecht ist.\nWenn du dich so benehst, kannst du deine eigenen Aussagen nicht bestätigen, was die Aussage deines Tweets erfordert. Es wäre hilfreich, wenn du dich selbst besser kennen würdest, damit du ein besseres Verständnis des eigenen Wesens hast und nicht nur von deinem Geschlecht abhängig bist.'